# The  Research  Space

"Here we use a large dataset of scholarly publications disambiguated at the individual level to create a map of science — or research  space — where links connect pairs of fields based on the probability that an individual has published in both of them."
[Original Article](https://arxiv.org/ftp/arxiv/papers/1602/1602.08409.pdf)

[Artigo suplementar](https://link.springer.com/content/pdf/10.1140/epjds/s13688-019-0210-z.pdf)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cmx
import networkx as nx
import pandas as pd
import numpy as np
import re

In [ ]:
plt.rcParams["figure.figsize"] = (20,10)

In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

### Our data

PRECISO CHECAR O PARSER NOVAMENTE. ALGUNS ANOS APARECERAM COMO 'rint' OU 'onic'. E EM ALGUNS ARTIGOS O NÚMERO DE AUTORES FOI ZERO.

Todas as publicações

In [ ]:
articles = pd.read_csv("../dataset/lattes_categories.csv", sep=";sep;", engine="python")
articles = articles[(articles["ano"] != 'rint') & (articles["ano"] != 'onic')]
articles.head()

Dados das categorias:

O dicionário abaixo é a classificação intermediária de acordo com o Scopus

In [ ]:
clss = dict()
clss[10] = "multidisciplinary"
clss[11] = "agricultural and biological sciences"
clss[12] = "arts and humanities"
clss[13] = "biochemistry, genetics and molecular biology"
clss[14] = "business, management and accounting"
clss[15] = "chemical engineering"
clss[16] = "chemistry"
clss[17] = "computer science"
clss[18] = "decision sciences"
clss[19] = "earth and planetary sciences"
clss[20] = "economics, econometrics and finance"
clss[21] = "energy"
clss[22] = "engineering"
clss[23] = "environmental science"
clss[24] = "immunology and microbiology"
clss[25] = "materials science"
clss[26] = "mathematics"
clss[27] = "medicine"
clss[28] = "neuroscience"
clss[29] = "nursing"
clss[30] = "pharmacology, toxicology and pharmaceutics"
clss[31] = "physics and astronomy"
clss[32] = "psychology"
clss[33] = "social sciences"
clss[34] = "veterinary"
clss[35] = "dentistry"
clss[36] = "health professions"

Podemos ler e adicionar uma nova coluna de classificação

In [ ]:
areas = pd.read_csv("../dataset/SJR/areas.txt", sep=";")
areas["Field"] = areas["Field"].apply(lambda x: x.strip().lower())
areas["Subject area"] = areas["Subject area"].apply(lambda x: x.strip().lower())
areas["Classification"] = areas["Code"].apply(lambda x: clss[int(str(x)[:2])])
areas.head()

Por fim, temos que adicionar algumas áreas que não foram cadastradas

In [ ]:
areas = areas.append({'Field': 'e-learning', 'Subject area': "social sciences & humanities",
                      "Classification": "social sciences"} , ignore_index=True)
areas = areas.append({'Field': 'nanoscience and nanotechnology', 'Subject area': "physical sciences",
                      "Classification": "materials science"} , ignore_index=True)
areas = areas.append({'Field': 'social work', 'Subject area': "social sciences & humanities",
                      "Classification": "social sciences"} , ignore_index=True)
areas = areas.append({'Field': 'sports science', 'Subject area': "health sciences",
                      "Classification": "health professions"} , ignore_index=True)

Instituição do pesquisador, caso queiramos agregar os dados

In [ ]:
bio = pd.read_csv("../dataset/lattes/pesquisadores.csv", sep=";sep;", engine="python")
bio.head()

### Filtering

"We filter this dataset by focusing only on scholars with less than fifty publications in each year, because those with more than fifty publications tend to have many publications that are miss-assigned and are not theirs"

VOU IGNORAR PORQUE NO LATTES É O USUÁRIO QUEM COLOCA.

### Research Space

" $\phi_{ff'}(T)$ is  the  adjacency  matrix  representing  the  research  space expressed  by  the  career trajectory of scientists in our dataset observed up to time T. "

Get all the categories from an article

In [ ]:
def catg(s):
    return [re.sub(r"\s?\(Q[1-9]\)", "", x).strip().lower() for x in s.split(";")]

Create a dict to represent the X matrix

In [ ]:
def X(t):
    x = dict()
    for _, row in articles.iterrows():
        if int(row["ano"]) < t:
            fs = catg(row["catg"])
            nf = len(fs)
            for field in fs:
                
                if row["num"] == 0:
                    continue
                
                if (row["pesq"], field) in x:
                    x[(row["pesq"], field)] += 1/(nf * row["num"])
                else:
                    x[(row["pesq"], field)] = 1/(nf * row["num"])
    
    print("X done")
    return x

Create a dict to represent the P matrix

In [ ]:
def P(t):
    x = X(t)
    p = dict()
    
    for sf in x:
        if x[sf] > 0.1:
            p[sf] = 1
            
    print("P done")
    return [p, x]

Create the M matrix

In [ ]:
def M(t):
    p, x = P(t)
    s = set()
    f = set()
    
    for sf in p:
        s.add(sf[0])
        f.add(sf[1])
        
    of = sorted(list(f))
    indices = {u: v for v, u in enumerate(of)}
    n = len(of)
    m = np.zeros((n,n))
    
    for i in range(len(of)):
        for j in range(i+1, len(of)):
            for k in s:
                if (k,of[i]) in p and (k,of[j]) in p:
                    m[i,j] += 1
                    m[j,i] += 1
                    
    print("M done")
    return [m, p, of, x]

Create the phi matrix

In [ ]:
def phi(t):
    m, p, of, x = M(t)
    indices = dict()
    sums = np.zeros(len(m))
    
    for sf in p:
        if sf[1] not in indices:
            indices[sf[1]] = of.index(sf[1])
        sums[indices[sf[1]]] += 1

    phi = m.copy()
    for i in range(len(m)):
        phi[:,i] /= sums[i]
    
    print("phi done")
    return [phi, of, x]

Salvar só pra ter certeza né. As etapas do X e do M são bastante demoradas.

In [ ]:
k, of, x = phi(2011)
np.save("../dataset/phi_matrix_2011.npy", k)
np.save("../dataset/x_dict_2011.npy", x)

with open("../dataset/of_2011.txt", "w") as f:
    for item in of:
        f.write("{}\n".format(item))

In [ ]:
k = np.load("../dataset/phi_matrix_2011.npy")
x = np.load("../dataset/x_dict_2011.npy", allow_pickle='TRUE').item()

of = list()
with open("../dataset/of_2011.txt", "r") as f:
    for item in f:
        of.append(item.strip())

In [ ]:
plt.imshow(k, cmap='hot', interpolation='nearest')
plt.show()

### Graph Plot

Get colors

In [ ]:
def get_colors(area, subs):
    values = [subs[area[node]] for node in of]

    cm = plt.get_cmap('gist_ncar')
    cNorm = colors.Normalize(vmin=0, vmax=max(values))
    scalar_map = cmx.ScalarMappable(norm=cNorm, cmap=cm)
    
    return [cm, scalar_map, values]

Get node size

In [ ]:
def get_node_size():
    values = [40 for node in of]
    values[of.index("computer science applications")] = 300    
    return values

Plot function

In [ ]:
def show_graph(A, area, subs, pos=None, threshold=0.212):
    G = nx.from_numpy_matrix(A)
    mast = nx.maximum_spanning_tree(G)
    
    for i in range(len(A)):
        for j in range(len(A)):
            if i != j:
                if A[i,j] > threshold:
                    mast.add_edge(i,j)
        
    if pos == None:
        pos = nx.spring_layout(mast)
    
    cm, scalarMap, values = get_colors(area, subs)
    
    f = plt.figure(1)
    ax = f.add_subplot(1,1,1)
    for label in subs:
        ax.plot([0],[0], color = scalarMap.to_rgba(subs[label]), label = label, lw=7)
        
    nx.draw_networkx(mast, pos, cmap=cm, vmin=0, vmax= max(values), node_color=values, with_labels=False,
                     ax=ax, node_size=get_node_size(), edge_size=1, edge_color="lightgray")
                                                                                                            
    plt.axis('off')
    f.set_facecolor('w')
    plt.legend(loc='upper left')
    plt.show()
    
    return pos

Using macro subjects

In [ ]:
dict_area = areas[["Field", "Subject area"]].set_index("Field").to_dict()["Subject area"]
unique = areas["Subject area"].unique()
subs = {u: v for v,u in enumerate(sorted(unique))}

pos = show_graph(k, dict_area, subs)

Podemos variar o valor do threshold para melhor visualizar

In [ ]:
def f(th):
    show_graph(k, dict_area, subs, pos, th)

interact(f, th=(0.05,0.95,0.05))

Using intermediate classification

In [ ]:
dict_area = areas[["Field", "Classification"]].set_index("Field").to_dict()["Classification"]
unique = areas["Classification"].unique()
subs = {u: v for v,u in enumerate(sorted(unique))}

pos = show_graph(k, dict_area, subs, pos)

### Revealed Comparative Advantage

"We next use the research space to predict the future presence of an individual, organization, or country in a research field. To make these predictions we define five possible states for individuals, organizations, or countries in a research field. These states are: inactive, active, nascent, intermediate, and developed."

In [ ]:
rca = dict()
sum_f = dict()
sum_s = dict()
sum_sf = 0

for sf in x:
    if sf[0] in sum_f:
        sum_f[sf[0]] += x[sf]
    else:
        sum_f[sf[0]] = x[sf]

    if sf[1] in sum_s:   
        sum_s[sf[1]] += x[sf]
    else:
        sum_s[sf[1]] = x[sf]
    
    sum_sf += x[sf]

    
for sf in x:
    rca[sf] = (x[sf]/sum_f[sf[0]])/(sum_s[sf[1]]/sum_sf)

Também podemos computar o RCA de instituições e de municípios/estados/regiões

In [ ]:
# TODO

### Plots:
    1. Áreas de maior atuação das principais instituições do país
    2. Mapa dos municípios/estados coloridos pelo RCA de uma área específica

E áreas urbanas

"First, we infer the country in which each affiliation-city pair is located; second, for each country, we compute a geographic distance matrix (using Vicenty’s formula) connecting each pair of cities; and lastly we use hierarchical clustering to define the different urban areas with the additional constraint that the maximum distance within each cluster has to be less than 50 km."